In [1]:
!pip install pandas
!pip install --upgrade jinja2
!pip install pip install openpyxl

In [2]:
%%html
<style> .container { width: 100% !important; }</style>

1. locations_df
2. Channelslocations_df
3. Channelslocations_pivoted_df
4. locations_prd_channels_df
5. [channel] names homonyms
6.  [channel] Number of channels to be migrated
7. [Locations] Number of locations per hvr meta database'
8. [Agents] Integrate remote Nodes
9. [Agents] Integrate remote Nodes
10. [Sources] Source Synonyms (ok for target , not ok for source- exception is file)
11. [Locations] homonymns on db_node_name  & db_instancelevels of locations
12. homonymns on db_node_name levels of locations

# common

In [1]:
import pandas as pd
pd.set_option('max_colwidth', 200)  # Set a very large value
pd.set_option('display.max_rows', 200) # Set a very large value

def add_borders(val):
    return 'border: 1px solid black; font-family: Consolas; font-size: 12px;'




style = lambda df : df.style.set_table_styles([
    {'selector': 'thead',
     'props': [('background-color', 'black'), ('border', '2px solid white'),('color','white')]}  ,
    {'selector': 'tr',
     'props': [ ('border', '1px solid black')]},
    {'selector': 'td',
     'props': [('background-color', 'lightgray'), ('border', '1px solid black')]}, 
])


style_df = lambda df : df.style.map(add_borders)

def get_dataframe_name(df):
    for name, obj in globals().items():
        if obj is df and isinstance(obj, pd.DataFrame):
            return name


getcols  = lambda df : display(style_df(pd.DataFrame (list(df.columns), columns=[get_dataframe_name(df)]).T))

def get_dataframe_columns():
    # Initialize an empty list to store the data
    data = []

    # Iterate through all variables in the global namespace
    for name, obj in globals().items():
        if isinstance(obj, pd.DataFrame):
            # Get the dataframe's column names
            columns = obj.columns.tolist()
            # Create a list of tuples containing df_name and col_name
            df_col_pairs = [(name, col) for col in columns]
            data.extend(df_col_pairs)

    # Create a new DataFrame from the collected data
    df = pd.DataFrame(data, columns=['df_name', 'col_name'])
    df['val'] = df['col_name']
    df=df[df.df_name.apply(lambda x : str(x).split('_')[-1])=="df" ]
    return df.pivot_table ( index = 'df_name', columns="col_name" , values = "val" , aggfunc = lambda x : ','.join(list (x)) ).fillna("")


--------
# Load Data

## locations_df

In [2]:
locations_df = pd.read_json("./data/LocationsParsed.json")
locations_prd_df = locations_df[locations_df['host'].isin(['odp-fin-prod-hvr-metadata.odp.health.ge.com','odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com'])]
getcols(locations_df)

,0,1,2,3,4,5,6,7,8,9,10,11
locations_df,database,db_instance,db_node_name,dbdesc,host,loc_class,loc_db_user,loc_description,loc_name,loc_remote_login,loc_remote_node,loc_remote_port


In [3]:
locations_df.head(1).T

,0
database,ushvr12
db_instance,usinnovationredshift
db_node_name,us-innovation-redshift.c8ziwm1qxh67.us-east-1.redshift.amazonaws.com
dbdesc,dev
host,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
loc_class,redshift
loc_db_user,504005930
loc_description,tredp
loc_name,tredp
loc_remote_login,isc_dev


## Table_agg_df

In [4]:
Table_agg_df = pd.read_json("./data/hvr_Table_Agg.json")

Table_agg_df['tbl_name_cnt'] = Table_agg_df['tbl_name_lst'].apply(len)
Table_agg_df['tbl_base_name_cnt'] = Table_agg_df['tbl_base_name_lst'].apply(len)

Table_agg_prd_df = Table_agg_df[Table_agg_df['host'].isin(['odp-fin-prod-hvr-metadata.odp.health.ge.com','odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com'])]
getcols(Table_agg_df),len(Table_agg_df)

,0,1,2,3,4,5,6,7
Table_agg_df,chn_name,tbl_name_lst,tbl_base_name_lst,host,database,dbdesc,tbl_name_cnt,tbl_base_name_cnt


(None, 2291)

In [5]:
Table_agg_df.head(2).T

,0,1
chn_name,edhr_01,bsr_fr_test
tbl_name_lst,"[lot, part]",[glb_bs_test]
tbl_base_name_lst,"[lot, part]",[glb_bs_test]
host,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com,odp-fin-prod-hvr-metadata.odp.health.ge.com
database,ushvr12,ushvr00
dbdesc,dev,fin-prod
tbl_name_cnt,2,1
tbl_base_name_cnt,2,1


## Channelslocations_df

In [6]:
Channelslocations_df = pd.read_json("./data/hvr_Channel2Location.json")
Channelslocations_prd_df = Channelslocations_df[Channelslocations_df['host'].isin(['odp-fin-prod-hvr-metadata.odp.health.ge.com','odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com'])]
getcols(Channelslocations_df)

,0,1,2,3,4,5,6,7
Channelslocations_df,chn_name,act_name,act_parameters,loc_class,loc_name,host,database,dbdesc


In [7]:
Channelslocations_df.head(2).T

,0,1
chn_name,edhr_01,edhr_01
act_name,Capture,Integrate
act_parameters,/NoTruncate,"/RenameExpression=""edhr_test/public/{hvr_tbl_name}/year={hvr_integ_tstamp %Y}/month={hvr_integ_tstamp %m}/day={hvr_integ_tstamp %d}/{hvr_integ_tstamp}.parquet"" /MaxFileSize=999999999"
loc_class,oracle,file
loc_name,sedhp,ts3ui
host,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com
database,ushvr12,ushvr12
dbdesc,dev,dev


## ChannelsLocationsTables_df

In [8]:
ChannelsLocationsTables_df = pd.merge(Channelslocations_df,Table_agg_df  , on=['database','host','chn_name'], how='left')
getcols(ChannelsLocationsTables_df)

#vierify join
len (Channelslocations_df),len(Table_agg_df),len(ChannelsLocationsTables_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12
ChannelsLocationsTables_df,chn_name,act_name,act_parameters,loc_class,loc_name,host,database,dbdesc_x,tbl_name_lst,tbl_base_name_lst,dbdesc_y,tbl_name_cnt,tbl_base_name_cnt


(11300, 2291, 11300)

## ChannelsLocationsTables_pivoted_df

In [9]:
ChannelsLocationsTables_pivoted_df.head().T

NameError: name 'ChannelsLocationsTables_pivoted_df' is not defined

In [10]:
#-- ignore all channels with d_ preffix
#-- count channels per action and pivot

joinNotEmpty = lambda column_name : '.'.join(list (filter (  lambda x : x!='' , column_name )))

lst = lambda x : list (x)
ChannelsLocationsTables_pivoted_df     = ChannelsLocationsTables_df[ChannelsLocationsTables_df.chn_name.apply(lambda x: x.split('_')[0]!='d')].pivot_table(index=['database','host','loc_name'] , columns='act_name', aggfunc={'tbl_base_name_cnt':['sum',len],'tbl_base_name_lst':lst ,'chn_name':lst}   ).fillna(0).reset_index()
ChannelsLocationsTables_pivoted_df.columns=list (map  ( joinNotEmpty ,  ChannelsLocationsTables_pivoted_df.columns))
ChannelsLocationsTables_pivoted_df.rename(columns={'tbl_base_name_cnt.sum.Capture'   : 'CaptureTables_cnt',
                                                   'tbl_base_name_cnt.sum.Integrate' : 'IntegrateTables_cnt',
                                                   'tbl_base_name_cnt.len.Capture'   : 'CaptureChn_cnt',
                                                   'tbl_base_name_cnt.len.Integrate' : 'IntegrateChn_cnt',
                                                   'tbl_base_name_lst.<lambda>.Capture'  : 'CaptureTables_lst',
                                                   'tbl_base_name_lst.<lambda>.Integrate': 'IntegrateTables_lst',
                                                   'chn_name.<lambda>.Capture': 'CaptureChn_lst',
                                                   'chn_name.<lambda>.Integrate': 'IntegrateChn_lst'
                                                  },inplace=True)

ChannelsLocationsTables_pivoted_prd_df =  ChannelsLocationsTables_pivoted_df[ChannelsLocationsTables_pivoted_df['host'].isin(['odp-fin-prod-hvr-metadata.odp.health.ge.com','odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com'])]
getcols(ChannelsLocationsTables_pivoted_prd_df)


,0,1,2,3,4,5,6,7,8,9,10
ChannelsLocationsTables_pivoted_prd_df,database,host,loc_name,CaptureChn_lst,IntegrateChn_lst,CaptureChn_cnt,IntegrateChn_cnt,CaptureTables_cnt,IntegrateTables_cnt,CaptureTables_lst,IntegrateTables_lst


#ChannelsLocationsTables_df[(ChannelsLocationsTables_df.tbl_name_cnt - ChannelsLocationsTables_df.tbl_base_name_cnt)>0] -- base name is same as tbl_name_cnt from count perspective
strdp = ChannelsLocationsTables_df[ChannelsLocationsTables_df.loc_name=='strdp']
strdp = strdp[strdp.chn_name.apply(lambda x: x.split('_')[0]!='d')]
strdp.pivot_table(index=['database','host','loc_class','loc_name'] , columns='act_name',values=['tbl_base_name_cnt'] , aggfunc=['sum',len]).fillna(0).reset_index()


In [11]:
#-- ignore all channels with d_ preffix
#-- count channels per action and pivot
Channelslocations_pivoted_df     = Channelslocations_df[Channelslocations_df.chn_name.apply(lambda x: x.split('_')[0]!='d')].pivot_table(index=['database','host','loc_name'] , columns='act_name',values='chn_name' , aggfunc=len).fillna(0).reset_index()
Channelslocations_pivoted_prd_df =  Channelslocations_pivoted_df[Channelslocations_pivoted_df['host'].isin(['odp-fin-prod-hvr-metadata.odp.health.ge.com','odp-us-prod-hvr-metadata-db.cjyhx859wwhg.us-east-1.rds.amazonaws.com'])]
getcols(Channelslocations_pivoted_df)
Channelslocations_pivoted_df.head()

,0,1,2,3,4
Channelslocations_pivoted_df,database,host,loc_name,Capture,Integrate


act_name,database,host,loc_name,Capture,Integrate
0,gla_poc,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com,alarm,1.0,0.0
1,gla_poc,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com,coret,1.0,0.0
2,gla_poc,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com,drm,3.0,0.0
3,gla_poc,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com,edhr,1.0,0.0
4,gla_poc,odp-us-innovation-postgres-db.cxzrm3enxxmy.us-east-1.rds.amazonaws.com,frevo,13.0,0.0


## locations_prd_channels_df
locations that are used on prod by channels not selected to be deleted

In [12]:
locations_prd_channels_df=pd.merge(locations_prd_df,Channelslocations_pivoted_prd_df  , on=['database','host','loc_name'], how='inner')
getcols(locations_prd_channels_df), len(locations_prd_channels_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
locations_prd_channels_df,database,db_instance,db_node_name,dbdesc,host,loc_class,loc_db_user,loc_description,loc_name,loc_remote_login,loc_remote_node,loc_remote_port,Capture,Integrate


(None, 548)

In [13]:
locations_channels_tables_prd_df=pd.merge(locations_prd_df,ChannelsLocationsTables_pivoted_df  , on=['database','host','loc_name'], how='inner')
getcols(locations_channels_tables_prd_df), len(locations_channels_tables_prd_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
locations_channels_tables_prd_df,database,db_instance,db_node_name,dbdesc,host,loc_class,loc_db_user,loc_description,loc_name,loc_remote_login,loc_remote_node,loc_remote_port,CaptureChn_lst,IntegrateChn_lst,CaptureChn_cnt,IntegrateChn_cnt,CaptureTables_cnt,IntegrateTables_cnt,CaptureTables_lst,IntegrateTables_lst


(None, 548)

# Sources summary

In [14]:
df = get_dataframe_columns().T.reset_index()

In [15]:
from itertools import product
from itertools import chain

pproduct = lambda x,y : product(y,[x])
denom_dict = lambda d : dict (chain.from_iterable(map( lambda k :list(pproduct(*(k))) ,  list(d.items())) ))

desc={ 'meta'    :['host','database','dbdesc'],
       'location':['loc_class','loc_remote_port','loc_description'],
       'agent'   :['loc_remote_login','loc_remote_node'],
       'action'  :['Capture','Integrate','loc_name','act_name','act_parameters'],
       'db'      :['loc_db_user','db_instance','db_node_name'],
       'channel' :['chn_name']
     }

desc_ = denom_dict(desc)

In [16]:
def get_df_plantuml ():
    df_columns = get_dataframe_columns().T.reset_index()   
    get_plantuml_object_str = lambda df : '\n'.join ( [f'object  {df}']+ [ f'{df} : {x} '  for x in sorted(set(df_columns[df])) ][1:] +['\n'])
    print ('\n'.join ([ get_plantuml_object_str(df) for df in list (get_dataframe_columns().reset_index()['df_name']) ] ))
 
#get_df_plantuml()

def get_df_dot ():
    df_columns = get_dataframe_columns().T.reset_index()   
    get_dot_object_str = lambda df : ''.join ( [f'{df} [label="{{ {{ {df} }}|{{']+ [ f'{x}|'  for x in sorted(set(df_columns[df])) ][1:] +['}}"]'])
    print ('\n'.join ([ get_dot_object_str(df) for df in list (get_dataframe_columns().reset_index()['df_name']) ] ))



#get_df_dot()



In [17]:
df['desc'] = df.col_name.apply ( lambda x : desc_.get(x,'_'))
style_df(df.set_index('col_name').sort_values('desc'))

df_name,ChannelsLocationsTables_df,ChannelsLocationsTables_pivoted_df,ChannelsLocationsTables_pivoted_prd_df,Channelslocations_df,Channelslocations_pivoted_df,Channelslocations_pivoted_prd_df,Channelslocations_prd_df,Table_agg_df,Table_agg_prd_df,locations_channels_tables_prd_df,locations_df,locations_prd_channels_df,locations_prd_df,desc
col_name,,,,,,,,,,,,,,
tbl_name_lst,tbl_name_lst,,,,,,,tbl_name_lst,tbl_name_lst,,,,,_
CaptureChn_cnt,,CaptureChn_cnt,CaptureChn_cnt,,,,,,,CaptureChn_cnt,,,,_
CaptureChn_lst,,CaptureChn_lst,CaptureChn_lst,,,,,,,CaptureChn_lst,,,,_
CaptureTables_cnt,,CaptureTables_cnt,CaptureTables_cnt,,,,,,,CaptureTables_cnt,,,,_
CaptureTables_lst,,CaptureTables_lst,CaptureTables_lst,,,,,,,CaptureTables_lst,,,,_
tbl_base_name_lst,tbl_base_name_lst,,,,,,,tbl_base_name_lst,tbl_base_name_lst,,,,,_
IntegrateChn_cnt,,IntegrateChn_cnt,IntegrateChn_cnt,,,,,,,IntegrateChn_cnt,,,,_
IntegrateChn_lst,,IntegrateChn_lst,IntegrateChn_lst,,,,,,,IntegrateChn_lst,,,,_
IntegrateTables_cnt,,IntegrateTables_cnt,IntegrateTables_cnt,,,,,,,IntegrateTables_cnt,,,,_


---------------
# Analysis 
--------------

## [Sources] Basic : Capture Sources + channel counts

In [20]:
capture    = set(locations_channels_tables_prd_df[locations_channels_tables_prd_df.CaptureChn_cnt >0].loc_remote_node) 
integtrate = set(locations_channels_tables_prd_df[locations_channels_tables_prd_df.IntegrateChn_cnt >0].loc_remote_node)
integtrate.intersection(capture)

{'',
 '10.229.1.113',
 '10.229.2.236',
 '10.242.109.196',
 '10.242.109.242',
 '10.242.112.153'}

In [21]:
Integrage_loc_remote_node_prd_list =  sorted(list(filter(lambda x:x!='' , integtrate)))
Integrage_loc_remote_node_prd_list

['10.229.1.113',
 '10.229.2.236',
 '10.242.109.196',
 '10.242.109.242',
 '10.242.112.153']

In [22]:
locations_channels_tables_prd_df['isCap&Int']  = locations_channels_tables_prd_df.loc_remote_node.isin(Integrage_loc_remote_node_prd_list)

In [23]:
from collections import Counter

In [25]:
maxN = lambda l , n : list (filter ( lambda x: x[0]!='' , sorted ( dict (Counter (l)).items() , key =  lambda  x : x[1])[::-1])) [:n] # add also sorting by length if same values!!
max3 = lambda l : maxN (l,3)
max_ = lambda l : sorted ( dict (Counter (l)).items() , key =  lambda  x : x[1])[::-1][0][0]

br = lambda x , n : '</br>'.join( set(x)) if len(set(x)) < n else ('</br>'.join(list(set(x))[:n] ) + f' </br>(..{len(set(x))})' )
br10 = lambda x : br (x,10)


from itertools import chain



flat_list =lambda lst:  list(chain(*lst))
unFloat = lambda y : list(filter ( lambda x : type(x)!=float, y))

f1 = lambda x : max3 (Counter(flat_list(unFloat(flat_list(x)))))
f2 = lambda x : len(set(flat_list(unFloat(flat_list(x)))))

df  = locations_channels_tables_prd_df [locations_channels_tables_prd_df.CaptureChn_cnt>0].groupby(['loc_class','db_node_name','isCap&Int']).agg({   'CaptureChn_cnt':[('chn_cnt','sum')] ,
                                                                                                                                                     'CaptureTables_cnt':[('table_cnt','sum')] ,
                                                                                                                                                     'CaptureTables_lst' :[('CaptureTables_cntr',f1),('CaptureTables_ucnt',f2),('CaptureTables_lst',list)],
                                                                                                                                                     'CaptureChn_lst' : [('CaptureChn_lst',list)], 
                                                                                                                                                     'db_instance':[('db_instance_cnt',lambda x : len(set(x)))],
                                                                                                                                                     'dbdesc' :[('dbdesc',lambda x : '</br>'.join(set(x)))],
                                                                                                                                                     'loc_name' :[('loc_names',br10)],
                                                                                                                                                     'database' :[('hub_cnt',len)],
                                                                                                                                                     'loc_description'   :[('loc_description_frq',max_)],#('loc_descriptions_wc3',max3)],
                                                                                                                                                     'loc_remote_node'   :[('loc_remote_nodes',br10)],
                                                                                                                                                     
                                                                                                                                                 }).sort_values([ 'loc_class',('CaptureChn_cnt','chn_cnt')], ascending = False)

df2 = pd.read_excel('./data/source_lkp.xlsx').astype(str)

columns_lst = ['loc_class', 'db_node_name', 'isCap&Int', 'chn_cnt', 'table_cnt','db_instance_cnt','hub_cnt','dbdesc', 'loc_description_frq','loc_remote_nodes','loc_names','type','DB_name','Support_Lead','comment']
columns_lst = ['loc_class', 'db_node_name', 'isCap&Int', 'chn_cnt', 'table_cnt','CaptureTables_lst','CaptureChn_lst']
#-- to flatten Multiindex for merge

takeLastNotEmpty = lambda column_name : list (filter (  lambda x : x!='' , column_name ))[-1]
ddf = df.reset_index()
ddf.columns = list (map  ( takeLastNotEmpty ,  df.reset_index().columns))
dddf = pd.merge(ddf, df2, left_on='db_node_name', right_on='Host',how='outer')[columns_lst].fillna('').set_index(['loc_class','db_node_name','isCap&Int'])
style (
    dddf.astype(str).apply(lambda x: x.str.rstrip('.'))
)

#-- to flatten Multiindex for merge

## channel Capture synonyms 

In [45]:
from functools import reduce 
chain = lambda l : list (reduce ( lambda x , y:  x+y , l))

x = chain ( dddf.reset_index()[dddf.reset_index()['db_node_name']=='gprs-dev-01-s1T.mgmt.cloud.ds.ge.com,2433'].CaptureTables_lst.values.tolist()[0])
y = chain (dddf.reset_index()[dddf.reset_index()['db_node_name']=='gprs-dev-01-s1T.mgmt.cloud.ds.ge.com,2433'].CaptureChn_lst.values.tolist()[0])
df0 = pd.DataFrame(list(zip(y,x)),columns = ['Channel','CaptureTable']).fillna('nan')
df=df0.explode('CaptureTable', ignore_index=True).fillna('')
df['val'] = df['CaptureTable']
#display(df0)
#display(df)
style(df.pivot_table( index = 'Channel', columns='CaptureTable',values='val', aggfunc='max').fillna(''))

CaptureTable,contingency_plan,good_onhand_extract,nan
Channel,,,
dbo_inno,contingency_plan,good_onhand_extract,
dbo_inno1,contingency_plan,,
dbo_inno2,contingency_plan,,
mapp_02,,,nan
mapp_04,contingency_plan,good_onhand_extract,
mapp_05,contingency_plan,,


-------------

Integrage_loc_remote_node_prd_list =  sorted(list(filter(lambda x:x!='' , set(locations_prd_channels_df[locations_prd_channels_df.Integrate >0].loc_remote_node))))
Integrage_loc_remote_node_prd_list

 set (Integrage_loc_remote_node_prd_list) -  {'10.229.1.113','10.242.109.196','10.242.109.242'}

locations_prd_channels_df['isCap&Int']  = locations_prd_channels_df.loc_remote_node.isin(Integrage_loc_remote_node_prd_list)

from collections import Counter

#TODO : add number of tables!!

maxN = lambda l , n : list (filter ( lambda x: x[0]!='' , sorted ( dict (Counter (l)).items() , key =  lambda  x : x[1])[::-1])) [:n] # add also sorting by length if same values!!
max3 = lambda l : maxN (l,3)
max_ = lambda l : sorted ( dict (Counter (l)).items() , key =  lambda  x : x[1])[::-1][0][0]

br = lambda x , n : '</br>'.join( set(x)) if len(set(x)) < n else ('</br>'.join(list(set(x))[:n] ) + f' </br>(..{len(set(x))})' )
br10 = lambda x : br (x,10)


df  = locations_prd_channels_df [locations_prd_channels_df.Capture>0].groupby(['loc_class','db_node_name','isCap&Int']).agg({'Capture':[('chn_cnt','sum')] ,
                                                                                                                 'db_instance':[('db_instance_cnt',lambda x : len(set(x)))],
                                                                                                                 'dbdesc' :[('dbdesc',lambda x : '</br>'.join(set(x)))],
                                                                                                                 'loc_name' :[('loc_names',br10)],
                                                                                                                 'database' :[('hub_cnt',len)],
                                                                                                                 'loc_description'   :[('loc_description_frq',max_)],#('loc_descriptions_wc3',max3)],
                                                                                                                 'loc_remote_node'   :[('loc_remote_nodes',br10)],
                                                                                                                            }).sort_values([ 'loc_class',('Capture','chn_cnt')], ascending = False).astype(str).apply(lambda x: x.str.rstrip('.0'))

df2 = pd.read_excel('./data/source_lkp.xlsx').astype(str)

columns_lst = ['loc_class', 'db_node_name', 'isCap&Int', 'chn_cnt', 'db_instance_cnt','hub_cnt','dbdesc', 'loc_description_frq','loc_remote_nodes','loc_names','type','DB_name','Support_Lead','comment']
#columns_lst = ['loc_class', 'db_node_name', 'isCap&Int', 'chn_cnt', 'db_instance_cnt','hub_cnt','dbdesc', 'loc_description_frq','loc_remote_nodes','type','DB_name','Support_Lead']

#-- to flatten Multiindex for merge

takeLastNotEmpty = lambda column_name : list (filter (  lambda x : x!='' , column_name ))[-1]
ddf = df.reset_index()
ddf.columns = list (map  ( takeLastNotEmpty ,  df.reset_index().columns))

style (
    pd.merge(ddf, df2, left_on='db_node_name', right_on='Host',how='outer')[columns_lst].fillna('').set_index(['loc_class','db_node_name','isCap&Int'])
)

df.style.set_table_styles([
    {'selector': 'thead',
     'props': [('background-color', 'black'), ('border', '2px solid white'),('color','white')]}  ,
    {'selector': 'tr',
     'props': [ ('border', '1px solid black')]},
    {'selector': 'td',
     'props': [('background-color', 'lightgray'), ('border', '1px solid black')]}, 
])

locations_prd_channels_df [locations_prd_channels_df.loc_class=='file_s3s']

## [channel] names homonyms
chanel with same name on different hubs will cause risk of overwirtting if we try to colocate them in same hub

In [ ]:
## TODO: group hubs RI / OPS

In [261]:
infra_lkp = pd.read_excel('./data/infra_lkp.xlsx').astype(str)

In [272]:
df = pd.merge(Channelslocations_prd_df, infra_lkp [['dbdesc','database','Track']] , on=['dbdesc','database'], how='left')

In [275]:
dff = df.groupby(['host','database','chn_name','Track'])[['loc_name']].count().reset_index()
style(
dff.pivot_table(index=['host','chn_name'],columns=['Track','database'] , values='loc_name' , aggfunc=len,margins=True).fillna('').astype(str).apply(lambda x: x.str.rstrip('.0')).sort_values(['All','host'] , ascending=False).head(84)
    )

##  [channel] Number of channels to be migrated

In [164]:
len(set (map (lambda x :','.join(x) , Channelslocations_prd_df[['host','database','chn_name']].values.tolist() )))   

1634

--## old defition locations_prod_df = locations_df[locations_df['dbdesc'].isin(['US hvr prod','fin-prod'])]

-- how many locations are using our agents?

In [165]:
len(list (map (   lambda x : ",".join(x), Channelslocations_pivoted_df[['host','database','loc_name']].values.tolist())))

1023

In [166]:
len(set (map (   lambda x : ",".join(x), locations_prd_df[['database','host','loc_name']].values.tolist())))

1878

In [20]:
len(Channelslocations_pivoted_df) , len(locations_prod_df) # --- are locations repated ? why numbers are different ???

(1023, 1878)

## [Locations] Number of locations per hvr meta database'

In [181]:
style_df(locations_prd_channels_df.groupby(['dbdesc','host','database'])['loc_name'].agg(len).reset_index().set_index(['dbdesc','database']))

print ('Number of locations per hvr meta database')
locations_prod_df.groupby(['dbdesc','host','database'])['loc_name'].agg(len).reset_index().set_index(['dbdesc','database'])

---------  if channel is using single agent for Capture and Single ganent for Integrate ( is is not matter if it is same agent or not ) total number of counts should be same !!

## [Agents] Integrate remote Nodes

In [183]:
df2= locations_prd_channels_df.pivot_table(index=['loc_remote_node','loc_class'], columns=['loc_remote_port'], values='Integrate', aggfunc='sum' , margins=True).fillna('_').astype(str).apply(lambda x: x.str.rstrip('.0'))
style_df(
    df2 
)

## [Agents] Integrate remote Nodes

In [185]:
df2= locations_prod_channels_df.pivot_table(index=['loc_remote_node','loc_class'], columns=['loc_remote_port'], values='Capture', aggfunc='sum',margins=True).fillna('_').astype(str).apply(lambda x: x.str.rstrip('.0'))
style_df(
    df2 
)

In [186]:
print ('number of agents per node (represented as port) - count of locations')

agg_function= lambda x : '<br>'.join(  set(x)) 
df1= locations_prd_df.groupby('loc_remote_node').agg({'database':agg_function})

df2= locations_prd_df.pivot_table(index=['loc_remote_node'], columns=['loc_remote_port'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
style_df(
#pd.merge(df1, df2, on='loc_remote_node', how='left')
df2   
    )

number of agents per node (represented as port) - count of locations


loc_remote_port,0,4343,4344,5454,6080,8080,8088,8443,9090,9995,18081
loc_remote_node,,,,,,,,,,,
,278,,,,,,,,,,
10.119.217.1,,1,,,,,,,,,
10.229.1.113,,1,7,,,,,,,,
10.229.2.236,,,,,,4,,,,,
10.242.109.196,,539,,,9,7,,13,,1,
10.242.109.235,,,,,,4,,,,,
10.242.109.242,,,,,,,,5,3,5,1
10.242.109196,,4,,,,,,,,,
10.242.112.130,,46,,1,,,,,,,


In [ ]:
-- rac://cgehedq2-scan.am.health.ge.com:1525 repeated multiple times is it same agent ?

In [29]:
style_df (
            locations_prod_df.pivot_table(index='loc_remote_node', columns=['loc_class'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
         )

loc_class,file,file_locdir,file_s3s,file_sftp,greenplum,kafka,mysql,oracle,postgresql,redshift,salesforce,sqlserver,teradata
loc_remote_node,,,,,,,,,,,,,
,,11,102,7,,4,2,111,31,,4,,6
10.119.217.1,,,,,,,,,,,,1,
10.229.1.113,,2,4,,,,,,,2,,,
10.229.2.236,,,2,,,,,,,2,,,
10.242.109.196,5,368,8,18,2,5,,49,58,35,,,21
10.242.109.235,,,,,,,,,,4,,,
10.242.109.242,,,15,,,,,,21,5,,,
10.242.109196,,4,,,,,,,,,,,
10.242.112.130,,,,,,,,,1,,3,43,


In [30]:
style_df (
    locations_prod_df.pivot_table(index='loc_remote_node', columns=['dbdesc','database'], values='loc_name', aggfunc='count').fillna('').astype(str).apply(lambda x: x.str.rstrip('.0'))
)

In [31]:
pd.DataFrame (locations_df.columns, columns=["columns"])

,columns
0,database
1,db_instance
2,db_node_name
3,dbdesc
4,host
5,loc_class
6,loc_db_user
7,loc_description
8,loc_name
9,loc_remote_login


for loc_lass in set(locations_prod_df['loc_class']):
    df = locations_prod_df[locations_prod_df['loc_class']==loc_lass]
    display(df)

## [Sources] Source Synonyms (ok for target , not ok for source- exception is file)

In [33]:
#//TODO filter singletons

concat_func = lambda x: '</br> '.join(set(map(str, x)))

agg_funcs = {'loc_name': concat_func }


for loc_lass in set(locations_prod_df['loc_class']):

    df = locations_prod_df[locations_prod_df['loc_class']==loc_lass]

    dff = style_df (
              df.pivot_table(index=['loc_class','db_node_name','db_instance','loc_db_user'], columns=['dbdesc'], values=['loc_name'], aggfunc=concat_func).fillna('').astype(str).apply(lambda x: x.str.rstrip('.0')).sort_values(by=['db_node_name'])
            )
    display (dff)

,,,,loc_name
,,,dbdesc,US hvr prod
loc_class,db_node_name,db_instance,loc_db_user,
file,###,###,###,az_lb ofa suath suate az_hf


In [ ]:
-- homonyms -> any locations that exists on different databases but has different meaning
-- should we do loc clean-up before or during migration (or after)
-- are loc_names unique?

## [Locations] homonymns on db_node_name  & db_instancelevels of locations

In [36]:
concat_func = lambda x :  [len (set(x)) , set(x)]

# Define a filtering function


dff2 = locations_prod_df.groupby(['dbdesc','loc_class','loc_name'])['database'].agg(lambda x : '<br>'.join(set(x)))

def filter_func(x):
    return x['Value'].sum() > 1

locations_prod_df.loc[:, 'db_concat'] = locations_prod_df['db_node_name'] + ' ---> ' + locations_prod_df['db_instance']


df= locations_prod_df.pivot_table(index=['dbdesc','loc_class','loc_name'], values=['db_concat'], aggfunc= concat_func ).fillna('').reset_index()#.astype(str).apply(lambda x: x.str.rstrip('.0'))
dff = df [df.db_concat.apply(lambda x : x[0] >1)].reset_index()
dff.loc[:, 'db_node_name_hmns'] = dff.apply(lambda x:  "</br>".join(x['db_concat'][1]) , axis=1)


dfff = pd.merge(dff, dff2, on=['dbdesc','loc_class','loc_name'], how='inner')


dfff.to_json('location_homonyms.json', orient='records', lines=True)

style_df(
    dfff[['dbdesc','loc_class','loc_name','db_node_name_hmns','database']].set_index(['dbdesc','loc_class','loc_name'])
)

## homonymns on db_node_name levels of locations

concat_func = lambda x :  [len (set(x)) , set(x)]

# Define a filtering function
def filter_func(x):
    return x['Value'].sum() > 1


df= locations_prod_df.pivot_table(index=['dbdesc','loc_class','loc_name'], values=['db_node_name'], aggfunc= concat_func ).fillna('').reset_index()#.astype(str).apply(lambda x: x.str.rstrip('.0'))
dff = df [df.db_node_name.apply(lambda x : x[0] >1)].reset_index()
dff.loc[:, 'db_node_name_hmns'] = dff.apply(lambda x:  "</br>".join(x['db_node_name'][1]) , axis=1)

style_df(
    dff[['dbdesc','loc_class','loc_name','db_node_name_hmns',]].set_index(['dbdesc','loc_class','loc_name'])
)